# RM 훈련 데이터셋 생성

이 노트북은 Reward Model 훈련을 위한 세 가지 유형의 선호도 쌍 데이터셋을 생성합니다:

1. **SPF (Supervised Preference Dataset by Filtering)**: GPT-4o 루브릭 평가 기반
2. **IMP (Inter-Model Performance Preference Dataset)**: 모델 성능 차이 기반
3. **ICP (Intra-Model Contrastive Preference Dataset)**: 대조적 지문 생성 기반

각 데이터셋은 6개 루브릭(평가 지침 완전성, 핵심 주제 명확성, 참고 자료 기반성, 논리적 흐름 및 구조, 한국어 품질, L2 학습자 적합성)에 대해 생성됩니다.
- completeness_for_guidelines
- core_theme_clarity
- reference_groundedness
- logical_flow_and_structure
- korean_quality
- l2_learner_suitability

In [1]:
import sys
from pathlib import Path

# 프로젝트 경로 설정
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'modules'))
sys.path.append(str(Path.cwd().parent / 'utils'))

print("✅ 경로 설정 완료")

✅ 경로 설정 완료


In [ ]:
import json
import random
from datetime import datetime

from utils.rm_dataset_generator import (
    RMDatasetGenerator,
    create_spf_dataset,
    create_imp_dataset,
    create_icp_dataset
)

print("✅ RM 데이터셋 생성기 모듈 로드 완료")

In [ ]:
# 기본 설정
OPENAI_MODEL = 'gpt-4o'
TARGET_PAIRS_PER_RUBRIC = 50  # 테스트용으로 적게 설정
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
print(f'🎯 설정 완료:')
print(f'   OpenAI 모델: {OPENAI_MODEL}')
print(f'   루브릭당 목표 쌍 수: {TARGET_PAIRS_PER_RUBRIC}')
print(f'   랜덤 시드: {RANDOM_SEED}')

In [ ]:
# 샘플 기본 지문 데이터 생성 (실제로는 기존 생성된 지문을 사용)
sample_base_passages = [
    {
        'id': f'passage_{i:03d}',
        'text': f'이것은 샘플 지문 {i}입니다. 한국어로 작성된 교육용 텍스트로, 다양한 주제를 다루고 있습니다. 문법적으로 정확하고 논리적으로 구성되어 있으며, 교육 목적에 적합한 내용을 담고 있습니다. 이 지문은 학습자들이 한국어 읽기 능력을 향상시킬 수 있도록 도와줍니다.',
        'topic': f'주제_{i%10}',
        'source': 'sample_generation',
        'created_at': datetime.now().isoformat()
    }
    for i in range(1, 101)
 ]

sample_file_path = Path.cwd().parent / 'data' / 'base_passages' / 'sample_passages_100.json'
sample_file_path.parent.mkdir(exist_ok=True)

with open(sample_file_path, 'w', encoding='utf-8') as f:
    json.dump(sample_base_passages, f, ensure_ascii=False, indent=2)

print(f'📝 샘플 기본 지문 생성 완료: {len(sample_base_passages)}개')
print(f'💾 저장 위치: {sample_file_path}')

In [ ]:
print('🚀 SPF 데이터셋 생성 시작...')

spf_dataset, spf_files = create_spf_dataset(
    passages_file='sample_passages_100.json',
    target_pairs_per_rubric=TARGET_PAIRS_PER_RUBRIC,
    openai_model=OPENAI_MODEL
)

print('\n✅ SPF 데이터셋 생성 완료!')
print('📁 저장된 파일들:')
for rubric, file_path in spf_files.items():
    print(f'   {rubric}: {Path(file_path).name}')

In [ ]:
high_perf_passages = [
    {
        'id': f'high_perf_{i:03d}',
        'text': f'고성능 모델이 생성한 우수한 품질의 지문 {i}입니다. 문법적으로 정확하고, 주제에 충실하며, 논리적으로 잘 구성되어 있습니다. 표현이 자연스럽고 완전한 정보를 제공합니다.',
        'model': 'high_performance_7B',
        'rubric_achievement_rate': 0.98,
        'created_at': datetime.now().isoformat()
    }
    for i in range(1, 51)
 ]

low_perf_passages = [
    {
        'id': f'low_perf_{i:03d}',
        'text': f'저성능 모델이 생성한 지문 {i}입니다. 문법에 일부 오류가 있고, 주제에서 벗어나는 경우가 있으며, 논리적 구성이 부족할 수 있습니다.',
        'model': 'low_performance_3B',
        'rubric_achievement_rate': 0.40,
        'created_at': datetime.now().isoformat()
    }
    for i in range(1, 51)
 ]

high_perf_path = Path.cwd().parent / 'data' / 'base_passages' / 'high_performance_passages.json'
low_perf_path = Path.cwd().parent / 'data' / 'base_passages' / 'low_performance_passages.json'

with open(high_perf_path, 'w', encoding='utf-8') as f:
    json.dump(high_perf_passages, f, ensure_ascii=False, indent=2)

with open(low_perf_path, 'w', encoding='utf-8') as f:
    json.dump(low_perf_passages, f, ensure_ascii=False, indent=2)

print(f'📝 고성능 모델 지문: {len(high_perf_passages)}개')
print(f'📝 저성능 모델 지문: {len(low_perf_passages)}개')
print(f'💾 저장 완료')

print('🚀 IMP 데이터셋 생성 시작...')

imp_dataset, imp_files = create_imp_dataset(
    high_perf_file='high_performance_passages.json',
    low_perf_file='low_performance_passages.json',
    target_pairs_per_rubric=TARGET_PAIRS_PER_RUBRIC,
    openai_model=OPENAI_MODEL
)

print('\n✅ IMP 데이터셋 생성 완료!')
print('📁 저장된 파일들:')
for rubric, file_path in imp_files.items():
    print(f'   {rubric}: {Path(file_path).name}')

In [ ]:
print('🚀 ICP 데이터셋 생성 시작...')

icp_dataset, icp_files = create_icp_dataset(
    base_passages_file='sample_passages_100.json',
    target_pairs_per_rubric=TARGET_PAIRS_PER_RUBRIC,
    openai_model=OPENAI_MODEL
)

print('\n✅ ICP 데이터셋 생성 완료!')
print('📁 저장된 파일들:')
for rubric, file_path in icp_files.items():
    print(f'   {rubric}: {Path(file_path).name}')